In [ ]:
import pandas as pd
from pathlib import Path

from curation_tools.curation_tools import download_file, upload_parquet_to_bq, concatenate_parquet_files
from curation_tools.mavedb_curation_tools import process_mavedb_metadata, process_mavedb

/Users/zakirov/Documents/GitHub/PerturbationCatalogue/.venv/lib/python3.12/site-packages/pandera/_pandas_deprecated.py:157: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


## Download MaveDB dump

In [2]:
mavedb_dump_url = "https://zenodo.org/records/15653325/files/mavedb-dump.20250612164404.zip"
mavedb_dump_path = f"../Dump/{mavedb_dump_url.split('/')[-1]}"

In [4]:
download_file(mavedb_dump_url, mavedb_dump_path, overwrite=False, unarchive=True)

Archive:  ../Dump/mavedb-dump.20250612164404.zip
 extracting: ../Dump/main.json       
 extracting: ../Dump/LICENSE.txt     
 extracting: ../Dump/csv/urn-mavedb-00000001-a-4.scores.csv  
 extracting: ../Dump/csv/urn-mavedb-00000001-a-2.scores.csv  
 extracting: ../Dump/csv/urn-mavedb-00000001-a-3.scores.csv  
 extracting: ../Dump/csv/urn-mavedb-00000001-a-1.scores.csv  
 extracting: ../Dump/csv/urn-mavedb-00000001-b-1.scores.csv  
 extracting: ../Dump/csv/urn-mavedb-00000001-b-2.scores.csv  
 extracting: ../Dump/csv/urn-mavedb-00000001-c-1.scores.csv  
 extracting: ../Dump/csv/urn-mavedb-00000001-c-2.scores.csv  
 extracting: ../Dump/csv/urn-mavedb-00000001-d-1.scores.csv  
 extracting: ../Dump/csv/urn-mavedb-00000001-d-2.scores.csv  
 extracting: ../Dump/csv/urn-mavedb-00000002-a-2.scores.csv  
 extracting: ../Dump/csv/urn-mavedb-00000002-a-2.counts.csv  
 extracting: ../Dump/csv/urn-mavedb-00000002-a-1.scores.csv  
 extracting: ../Dump/csv/urn-mavedb-00000002-a-1.counts.csv  
 extrac

## Convert json to flat df

In [3]:
mavedb_metadata, processed_df = process_mavedb_metadata("../Dump/mavedb-dump.20250612164404/main.json")
mavedb_metadata.head()

# uncomment to generate template for manual curation
# mavedb_metadata.to_excel('../mavedb_studies_template.xlsx', index=False)

## Read the manually curated mavedb metadata file

In [5]:
mavedb_metadata_cur = pd.read_excel("../mavedb_studies.xlsx")
# filter for curated only
mavedb_metadata_cur = mavedb_metadata_cur[mavedb_metadata_cur['is_curated'] == 'Yes']

## Processing of a MaveDB entry

In [10]:
for mavedb_entry in mavedb_metadata_cur['dataset_id']:
    print(f"ℹ️ Processing: {mavedb_entry}")
    process_mavedb(
        mavedb_dataset_id=mavedb_entry,
        mavedb_csv_dir='../Dump/mavedb-dump.20250612164404/csv',
        curated_metadata_df=mavedb_metadata_cur,
        non_curated_h5ad_dir=Path("../non_curated/h5ad"),
        overwrite=False
    )

ℹ️ Processing: urn:mavedb:00000001-a-1
✅ Saved AnnData object to ../non_curated/h5ad/urn-mavedb-00000001-a-1.h5ad
Loading data from ../non_curated/h5ad/urn-mavedb-00000001-a-1.h5ad
Column perturbation_name has no NA entries in adata.obs
Mapped potential synonyms in perturbed_target_symbol of the provided dataframe to gene symbols


/var/folders/f_/7vds5wkn7vn5h991mk4_ybt80000gp/T/ipykernel_8482/2656960001.py:45: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  OBS_df = OBS_df.replace({None: np.nan})
/Users/zakirov/Documents/GitHub/PerturbationCatalogue/data_exploration/curation_tools/curation_tools.py:1516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  map_df["synonyms"] = map_df["synonyms"].str.split("|")


Converted 1/1 gene symbols/ENSG IDs to standardized gene symbols/ENSG IDs
--------------------------------------------------
Counted entries in column perturbed_target_symbol of adata.obs and stored in perturbed_target_number
Chromosome encoding applied to perturbed_target_chromosome in adata.obs and stored as 'perturbed_target_chromosome_encoding'.
Matched columns of adata.obs to the obs_schema.
adata.obs is valid according to the obs_schema.
✅ Curated h5ad data saved to ../curated/h5ad/urn-mavedb-00000001-a-1_curated.h5ad


/Users/zakirov/Documents/GitHub/PerturbationCatalogue/data_exploration/curation_tools/curation_tools.py:293: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  adata.obs = adata.obs.fillna(value=np.nan)


✅ Metadata saved to ../curated/parquet/urn-mavedb-00000001-a-1_curated_metadata.parquet
Processing data...
Saving data to ../curated/parquet/urn-mavedb-00000001-a-1_curated_data.parquet...
✅ Data saved to ../curated/parquet/urn-mavedb-00000001-a-1_curated_data.parquet
ℹ️ Processing: urn:mavedb:00000001-a-2
✅ Saved AnnData object to ../non_curated/h5ad/urn-mavedb-00000001-a-2.h5ad
Loading data from ../non_curated/h5ad/urn-mavedb-00000001-a-2.h5ad
Column perturbation_name has no NA entries in adata.obs
Mapped potential synonyms in perturbed_target_symbol of the provided dataframe to gene symbols


...

/Users/zakirov/Documents/GitHub/PerturbationCatalogue/data_exploration/curation_tools/curation_tools.py:293: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  adata.obs = adata.obs.fillna(value=np.nan)


Saving data to ../curated/parquet/urn-mavedb-00000001-d-1_curated_data.parquet...
✅ Data saved to ../curated/parquet/urn-mavedb-00000001-d-1_curated_data.parquet
ℹ️ Processing: urn:mavedb:00000001-d-2
✅ Saved AnnData object to ../non_curated/h5ad/urn-mavedb-00000001-d-2.h5ad
Loading data from ../non_curated/h5ad/urn-mavedb-00000001-d-2.h5ad
Column perturbation_name has no NA entries in adata.obs


/var/folders/f_/7vds5wkn7vn5h991mk4_ybt80000gp/T/ipykernel_8482/2656960001.py:45: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  OBS_df = OBS_df.replace({None: np.nan})
/Users/zakirov/Documents/GitHub/PerturbationCatalogue/data_exploration/curation_tools/curation_tools.py:1516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  map_df["synonyms"] = map_df["synonyms"].str.split("|")


Mapped potential synonyms in perturbed_target_symbol of the provided dataframe to gene symbols
Converted 1/1 gene symbols/ENSG IDs to standardized gene symbols/ENSG IDs
--------------------------------------------------
Counted entries in column perturbed_target_symbol of adata.obs and stored in perturbed_target_number
Chromosome encoding applied to perturbed_target_chromosome in adata.obs and stored as 'perturbed_target_chromosome_encoding'.
Matched columns of adata.obs to the obs_schema.
adata.obs is valid according to the obs_schema.
✅ Curated h5ad data saved to ../curated/h5ad/urn-mavedb-00000001-d-2_curated.h5ad
✅ Metadata saved to ../curated/parquet/urn-mavedb-00000001-d-2_curated_metadata.parquet
Processing data...
Saving data to ../curated/parquet/urn-mavedb-00000001-d-2_curated_data.parquet...
✅ Data saved to ../curated/parquet/urn-mavedb-00000001-d-2_curated_data.parquet


/Users/zakirov/Documents/GitHub/PerturbationCatalogue/data_exploration/curation_tools/curation_tools.py:293: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  adata.obs = adata.obs.fillna(value=np.nan)


## Concatenate all parquet files and save

In [11]:
parquet_dir = "../curated/parquet"
output_path_metadata = "../curated/mavedb_all_curated_metadata.parquet"
output_path_data = "../curated/mavedb_all_curated_data.parquet"

### Metadata

In [ ]:
concatenate_parquet_files(
    parquet_dir=parquet_dir,
    output_path=output_path_metadata,
    pattern="*_curated_metadata.parquet"
)

### Data

In [ ]:
concatenate_parquet_files(
    parquet_dir=parquet_dir,
    output_path=output_path_data,
    pattern="*_curated_data.parquet"
)

## Upload to BQ

In [13]:
bq_dataset_id = "prj-ext-dev-pertcat-437314.mavedb"
curated_parquet_data_path = "../curated/mavedb_all_curated_data.parquet"
curated_parquet_metadata_path = "../curated/mavedb_all_curated_metadata.parquet"
bq_metadata_table_name="metadata"
bq_data_table_name="data"

### Metadata

In [14]:
upload_parquet_to_bq(
    parquet_path=curated_parquet_metadata_path,
    bq_dataset_id=bq_dataset_id,
    bq_table_name=bq_metadata_table_name,
    key_columns=["dataset_id", "sample_id"]
)

Staging table: loading `.parquet` file ../curated/mavedb_all_curated_metadata.parquet to prj-ext-dev-pertcat-437314.mavedb.metadata_staging...
Staging table: loaded 54483 rows to prj-ext-dev-pertcat-437314.mavedb.metadata_staging
Staging table: added ingested_at timestamp column to prj-ext-dev-pertcat-437314.mavedb.metadata_staging
Merge completed: staging data merged into prj-ext-dev-pertcat-437314.mavedb.metadata
Staging table: deleted prj-ext-dev-pertcat-437314.mavedb.metadata_staging


### Data

In [16]:
upload_parquet_to_bq(
    parquet_path=curated_parquet_data_path,
    bq_dataset_id=bq_dataset_id,
    bq_table_name=bq_data_table_name,
    key_columns=["dataset_id", "sample_id"]
)

Staging table: loading `.parquet` file ../curated/mavedb_all_curated_data.parquet to prj-ext-dev-pertcat-437314.mavedb.data_staging...
Staging table: loaded 233437 rows to prj-ext-dev-pertcat-437314.mavedb.data_staging
Staging table: added ingested_at timestamp column to prj-ext-dev-pertcat-437314.mavedb.data_staging
Merge completed: staging data merged into prj-ext-dev-pertcat-437314.mavedb.data
Staging table: deleted prj-ext-dev-pertcat-437314.mavedb.data_staging
